In [80]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import tensorflow_hub as hub
import numpy as np
import tensorflow_text
import datetime as dt
import re


In [136]:
dftest = pd.read_json('/Users/ianndelap/code/ianndelap/market_sentiment/raw_data/dump.json',lines=True,nrows=20000)

"""
let's choose only the features we need
"""
df = dftest[['full_text',
'lang',
'datetimestamp',
'reply_count',
'retweet_count',
'quote_count',
'created_at',
'favorite_count',
'symbols',
'hashtags']]

In [137]:
def replacen(x):
    return x.replace('\n','')
def replacer(x):
    return x.replace('\r','')
def replacern(x):
    return x.replace('\r\n','')

df['full_text'] = df['full_text'].apply(replacer)
df['full_text'] = df['full_text'].apply(replacen)
df['full_text'] = df['full_text'].apply(replacern)


/var/folders/99/ckqc944s1nl57yz05vhcmt6w0000gn/T/ipykernel_63580/2141458383.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['full_text'] = df['full_text'].apply(replacer)
/var/folders/99/ckqc944s1nl57yz05vhcmt6w0000gn/T/ipykernel_63580/2141458383.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['full_text'] = df['full_text'].apply(replacen)
/var/folders/99/ckqc944s1nl57yz05vhcmt6w0000gn/T/ipykernel_63580/2141458383.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [138]:
# Intel
def tickerintc(x):
    if type(x) == list:
        newlist = []
        for item in x:
            newlist.append(item.lower())
        if 'intc' in newlist:
            return 1
        else:
            return 0
        
def hashtag_intc(x):
    if type(x) == list:
        newlist = []
        for item in x:
            newlist.append(item.lower())
        intc = ['intel','intc']
        if intc[0] in newlist or intc[1] in newlist:
            return 1
        else:
            return 0
        
# Beyond Meat
def tickerbynd(x):
    if type(x) == list:
        newlist = []
        for item in x:
            newlist.append(item.lower())
        if 'bynd' in newlist:
            return 1
        else:
            return 0
        
def hashtag_bynd(x):
    if type(x) == list:
        newlist = []
        for item in x:
            newlist.append(item.lower())
        bynd = ['beyondmeat','bynd']
        if bynd[0] in newlist or bynd[1] in newlist:
            return 1
        else:
            return 0
# General Electric
def tickerge(x):
    if type(x) == list:
        newlist = []
        for item in x:
            newlist.append(item.lower())
        if 'ge' in newlist:
            return 1
        else:
            return 0
        
def hashtag_ge(x):
    if type(x) == list:
        newlist = []
        for item in x:
            newlist.append(item.lower())
        ge = ['generalelectric','ge']
        if ge[0] in newlist or ge[1] in newlist:
            return 1
        else:
            return 0
        
# Bitcoin (  >:(   )   
def tickerbtc(x):
    if type(x) == list:
        newlist = []
        for item in x:
            newlist.append(item.lower())
        if 'btc' in newlist:
            return 1
        else:
            return 0
        
def hashtag_btc(x):
    if type(x) == list:
        newlist = []
        for item in x:
            newlist.append(item.lower())
        btc = ['bitcoin','btc']
        if btc[0] in newlist or btc[1] in newlist:
            return 1
        else:
            return 0

In [139]:
"""
create our new categorical columns regarding symbols (4x) and hashtags (4x)
"""
        
df['intc_ticker'] = df['symbols'].apply(tickerintc)        
df['bynd_ticker'] = df['symbols'].apply(tickerbynd)        
df['ge_ticker'] = df['symbols'].apply(tickerge)        
df['btc_ticker'] = df['symbols'].apply(tickerbtc)

df['#intc_ticker'] = df['hashtags'].apply(hashtag_intc)        
df['#bynd_ticker'] = df['hashtags'].apply(hashtag_bynd)        
df['#ge_ticker'] = df['hashtags'].apply(hashtag_ge)        
df['#btc_ticker'] = df['hashtags'].apply(hashtag_btc)

"""
fill nan values with zeros in our new columns
"""

df['intc_ticker'].fillna(0.,inplace=True)
df['bynd_ticker'].fillna(0.,inplace=True)
df['ge_ticker'].fillna(0.,inplace=True)
df['btc_ticker'].fillna(0.,inplace=True)

df['#intc_ticker'].fillna(0.,inplace=True)
df['#bynd_ticker'].fillna(0.,inplace=True)
df['#ge_ticker'].fillna(0.,inplace=True)
df['#btc_ticker'].fillna(0.,inplace=True)


/var/folders/99/ckqc944s1nl57yz05vhcmt6w0000gn/T/ipykernel_63580/2353844641.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['intc_ticker'] = df['symbols'].apply(tickerintc)
/var/folders/99/ckqc944s1nl57yz05vhcmt6w0000gn/T/ipykernel_63580/2353844641.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bynd_ticker'] = df['symbols'].apply(tickerbynd)
/var/folders/99/ckqc944s1nl57yz05vhcmt6w0000gn/T/ipykernel_63580/2353844641.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

In [140]:
df[['intc_ticker','bynd_ticker','ge_ticker','btc_ticker']].fillna(0.,inplace=True)
df[['#intc_ticker','#bynd_ticker','#ge_ticker','#btc_ticker']].fillna(0.,inplace=True)

/Users/ianndelap/.pyenv/versions/3.8.6/envs/market_sentiment/lib/python3.8/site-packages/pandas/core/frame.py:5171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [141]:
df['created_at'] = df['created_at'].dt.date

/var/folders/99/ckqc944s1nl57yz05vhcmt6w0000gn/T/ipykernel_63580/2687736359.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['created_at'] = df['created_at'].dt.date


In [142]:
df.drop(['favorite_count',
         'lang',
         'datetimestamp',
         'reply_count',
         'retweet_count',
         'quote_count',
         'quote_count',
         'symbols',
         'hashtags'],axis=1,inplace=True)


/Users/ianndelap/.pyenv/versions/3.8.6/envs/market_sentiment/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [143]:
def remove_urls(text):
    text = re.sub(r'https?://\S+', '', text)
    return text

df['full_text'] = df['full_text'].apply(remove_urls)

/var/folders/99/ckqc944s1nl57yz05vhcmt6w0000gn/T/ipykernel_63580/3722614064.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['full_text'] = df['full_text'].apply(remove_urls)


In [ ]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")
text_result = embed(df['full_text'])
vector_df = pd.DataFrame(text_result.numpy())

In [90]:
df = pd.concat([df,vector_df], axis=1, join="inner")

In [93]:
"""
select the dataframes for when the symbol tickers have a hit
"""

intc_ticker_df = df[df['intc_ticker']==1.]
bynd_ticker_df = df[df['bynd_ticker']==1.]
ge_ticker_df = df[df['ge_ticker']==1.]
btc_ticker_df = df[df['btc_ticker']==1.]

"""
select the dataframes for when the hashtag tickers have a hit
"""

#intc_hash_ticker_df = df[df['#intc_ticker']==1.]
#bynd_hash_ticker_df = df[df['#bynd_ticker']==1.]
#ge_hash_ticker_df = df[df['#ge_ticker']==1.]
#btc_hash_ticker_df = df[df['#btc_ticker']==1.]

'\nselect the dataframes for when the hashtag tickers have a hit\n'

In [95]:
intc_ticker_mean_df = intc_ticker_df.groupby('created_at').mean()
bynd_ticker_mean_df = bynd_ticker_df.groupby('created_at').mean()
ge_ticker_mean_df = ge_ticker_df.groupby('created_at').mean()
btc_ticker_mean_df = btc_ticker_df.groupby('created_at').mean()
                   
intc_hash_ticker_mean_df = intc_hash_ticker_df.groupby('created_at').mean()
bynd_hash_ticker_mean_df = bynd_hash_ticker_df.groupby('created_at').mean()
ge_hash_ticker_mean_df = ge_hash_ticker_df.groupby('created_at').mean()
btc_hash_ticker_mean_df = btc_hash_ticker_df.groupby('created_at').mean()

In [96]:
"""
convert created_at index from object to datetime 
"""
# symbol ticker dfs
intc_ticker_mean_df.index = pd.to_datetime(intc_ticker_mean_df.index)
bynd_ticker_mean_df.index = pd.to_datetime(bynd_ticker_mean_df.index)
ge_ticker_mean_df.index = pd.to_datetime(ge_ticker_mean_df.index)
btc_ticker_mean_df.index = pd.to_datetime(btc_ticker_mean_df.index)
# hashtag ticker dfs
intc_hash_ticker_mean_df.index = pd.to_datetime(intc_hash_ticker_mean_df.index)
bynd_hash_ticker_mean_df.index = pd.to_datetime(bynd_hash_ticker_mean_df.index)
ge_hash_ticker_mean_df.index = pd.to_datetime(ge_hash_ticker_mean_df.index)
btc_hash_ticker_mean_df.index = pd.to_datetime(btc_hash_ticker_mean_df.index)

# target

In [59]:
import yfinance as yf
tickers = yf.download(tickers = 'INTC BYND GE BTC', interval='1d',start="2018-01-01", end="2021-08-01")

[*********************100%***********************]  4 of 4 completed


In [97]:

INTCclose = tickers['Close']['INTC']
BYNDclose = tickers['Close']['BYND']
GEclose = tickers['Close']['GE']
BTCclose = tickers['Close']['BTC']

def onezero(x):
    if x>0:
        return 1
    else:
        return 0

intc_target_df = pd.Series(INTCclose).diff().apply(onezero)
bynd_target_df = pd.Series(BYNDclose).diff().apply(onezero)
ge_target_df = pd.Series(GEclose).diff().apply(onezero)
btc_target_df = pd.Series(BTCclose).diff().apply(onezero)

In [98]:
"""
Merge ticker feature selected aggredated dataframe with target target dataframe created
"""

INTC_df = pd.merge(intc_ticker_mean_df,intc_target_df,left_index=True,right_index=True)

In [99]:
"""
Merge ticker feature selected aggredated dataframe with target target dataframe created
"""

BYND_df = pd.merge(bynd_ticker_mean_df,bynd_target_df,left_index=True,right_index=True)

In [100]:
GE_df = pd.merge(ge_ticker_mean_df,ge_target_df,left_index=True,right_index=True)

In [101]:
BTC_df = pd.merge(btc_ticker_mean_df,btc_target_df,left_index=True,right_index=True)

In [102]:
GE_df

,intc_ticker,bynd_ticker,ge_ticker,btc_ticker,#intc_ticker,#bynd_ticker,#ge_ticker,#btc_ticker,0,1,...,503,504,505,506,507,508,509,510,511,GE
2018-05-09,0.000000,0.000000,1.0,0.000000,0.0,0.0,0.027027,0.0,-0.010631,0.021277,...,0.020193,-0.020014,-0.016109,0.058573,-0.021095,0.049080,0.008452,0.005866,-0.014512,1
2018-09-18,0.000000,0.000000,1.0,0.000000,0.0,0.0,0.111111,0.0,-0.009547,0.018435,...,0.015526,-0.026060,-0.003165,0.063126,-0.037541,0.029145,0.025474,-0.007508,-0.018474,0
2018-09-19,0.090909,0.000000,1.0,0.090909,0.0,0.0,0.090909,0.0,0.002870,0.026602,...,0.005754,-0.008333,0.002102,0.054130,-0.010791,0.029788,-0.011307,0.012566,0.041356,1
2019-06-14,0.166667,0.333333,1.0,0.250000,0.0,0.0,0.000000,0.0,-0.004222,-0.013141,...,-0.000486,-0.015445,-0.042445,0.045304,-0.006380,0.043819,-0.018205,-0.002486,-0.011861,0
2019-10-25,0.000000,0.200000,1.0,0.000000,0.0,0.0,0.400000,0.2,-0.026116,-0.019183,...,0.010279,-0.017124,-0.050103,0.066949,0.024303,0.020473,-0.018705,0.000735,0.018061,1
2021-04-13,0.000000,0.000000,1.0,0.000000,0.0,0.0,0.000000,0.0,-0.032445,-0.031735,...,0.026510,-0.033095,-0.019501,0.077242,-0.016935,-0.068614,0.022665,0.002388,0.003421,0
2021-04-14,0.000000,0.125000,1.0,0.000000,0.0,0.0,0.000000,0.0,0.010953,-0.014694,...,-0.004171,-0.008110,-0.047059,0.021465,-0.016335,0.009949,-0.014086,-0.019378,0.007075,1


In [29]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_validate


In [129]:
X= BTC_df.drop(columns=['BTC','#btc_ticker','#bynd_ticker','#intc_ticker','#btc_ticker','bynd_ticker','intc_ticker','#ge_ticker','ge_ticker'])
y = BTC_df['BTC']

In [130]:
X

,btc_ticker,0,1,2,3,4,5,6,7,8,...,502,503,504,505,506,507,508,509,510,511
2018-05-09,1.0,0.016767,0.001793,-0.080035,-0.075438,0.034375,0.027074,-0.032280,-0.008594,0.008797,...,-0.012638,-0.046418,0.000975,-0.068922,0.045940,0.013546,-0.014426,-0.016907,0.000310,0.033897
2018-09-18,1.0,0.016924,0.013106,-0.075544,-0.077373,0.037747,0.022004,-0.029426,0.002181,0.011508,...,-0.005065,-0.056240,0.005479,-0.072258,0.047200,0.019194,-0.018435,-0.016006,0.001576,0.024344
2018-09-19,1.0,0.015555,0.008971,-0.079529,-0.078244,0.038912,0.024849,-0.031229,-0.009196,0.003024,...,-0.014470,-0.054583,-0.003669,-0.067646,0.045440,0.018488,-0.018897,-0.017976,-0.004122,0.027814
2019-02-01,1.0,0.088351,0.054780,-0.079269,-0.086880,-0.010787,0.031329,-0.039677,0.050784,0.045990,...,-0.012000,-0.063440,0.076560,-0.079728,0.071660,-0.005419,0.050477,0.070211,0.042883,0.082415
2019-06-14,1.0,0.011015,0.010086,-0.079866,-0.077299,0.026303,0.029523,-0.020479,-0.001986,-0.003655,...,-0.019253,-0.050819,0.003333,-0.065242,0.046076,0.015910,-0.013867,-0.020349,-0.001548,0.034184
2019-10-25,1.0,0.010308,0.019097,-0.080394,-0.080023,0.023063,0.038322,-0.018535,0.002054,-0.011623,...,-0.024194,-0.048106,0.013497,-0.061101,0.043004,0.019125,-0.003985,-0.017919,0.002946,0.039238
2020-02-28,1.0,-0.022626,-0.062570,-0.074158,-0.060570,0.030903,0.038593,0.013535,0.012855,0.008787,...,-0.033491,-0.017612,0.029001,-0.059762,0.038988,0.035195,0.052776,-0.076614,0.023414,0.054135
2020-03-03,1.0,0.039671,-0.070008,-0.075505,-0.075719,0.064634,0.043652,-0.005378,0.006514,0.034962,...,-0.001540,-0.074560,-0.022291,-0.075621,0.069339,0.049339,-0.059214,0.048741,-0.012951,-0.000595
2020-03-04,1.0,0.061630,-0.054590,-0.091329,-0.090509,-0.008032,0.015823,-0.046515,-0.001367,0.047693,...,0.015616,-0.035481,-0.013759,-0.077884,0.059235,0.021716,0.026960,0.012880,0.006999,0.038281
2020-03-05,1.0,0.055514,0.044088,-0.088157,-0.067306,-0.016000,0.036814,-0.023541,-0.074903,0.020206,...,-0.045300,0.007771,0.059201,-0.033190,0.041460,0.007425,0.038917,-0.004413,0.005572,0.045212


In [134]:
tree = DecisionTreeRegressor()
cv_results = cross_validate(tree, X, y, scoring = "r2", cv=5)
cv_results['test_score'].mean()
#'std: ', cv_results['test_score'].std())
#print('mean: ',cv_results['test_score'].mean())

-0.2

In [135]:
cv_results

{'fit_time': array([0.00829792, 0.01333594, 0.02683687, 0.01223087, 0.0437603 ]),
 'score_time': array([0.00893593, 0.01360106, 0.02147818, 0.03989387, 0.03128695]),
 'test_score': array([ 1.,  0.,  0., -1., -1.])}

In [ ]:
pipe_tree = Pipeline([('tree', DecisionTreeRegressor())
    
])

In [ ]:
cv_results = cross_validate(pipe_tree, X, y, scoring = rmsle, cv=10)

In [ ]:
from sklearn.model_selection import GridSearchCV

# Instanciate grid search to find the best hyperparams for the Ridge()
grid_search = GridSearchCV(
    pipe_baseline, 
    param_grid={
        # Access any component of the pipeline, as far back as you want
        'linear_regression__alpha': [1,5,10,50],
        'linear_regression__solver': ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']},
    cv=5,
    scoring=rmsle_neg)